## Import

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


### 1 - Télécharger le fichier alice.txt depuis l’espace de cours. Charger ce fichier en python et afficher son contenu.

In [37]:
alice_text = open("alice.txt", encoding='utf-8').read()

### 2 - Depuis la chaîne de caractères précédemment chargée, générer la liste de caractères uniques présents dans le livre. Écrire une méthode permettant de transformer chaque caratère en entier et inversement (chaque entier en caractère). Ceci est nécessaire car les réseaux de neurones ne travaillent pas sur des caractères mais sur des nombres !

In [38]:
unique_car = set()

for letter in alice_text:
    unique_car.add(letter)

In [39]:
car_to_int = {}
int_to_car = {}

for i, c in enumerate(unique_car):
    car_to_int[c] = i
    int_to_car[i] = c

### 3 - Préparer un dataset contenant des séquences de 50 caractères (paramétrable) de longueur en entrée et le caractère suivant cette séquence en target. 

In [40]:
def create_dataframe(data, decalage):
    X, y = [], []
    for i in range(len(data) - decalage):
        X.append(data[i:i+decalage])
        y.append(car_to_int[data[i+decalage]])
    return np.array(X), np.array(y)

In [41]:
decalage = 50
data_X, data_y = create_dataframe(alice_text, decalage)

In [42]:
df = pd.DataFrame({'x':data_X.tolist(), 'y':data_y})

In [43]:
df

,x,y
0,﻿The Project Gutenberg eBook of Alice’s Advent...,18
1,The Project Gutenberg eBook of Alice’s Adventu...,30
2,he Project Gutenberg eBook of Alice’s Adventur...,58
3,e Project Gutenberg eBook of Alice’s Adventure...,18
4,Project Gutenberg eBook of Alice’s Adventures...,62
...,...,...
163961,ibe to our email newsletter to hear about new ...,55
163962,be to our email newsletter to hear about new e...,12
163963,e to our email newsletter to hear about new eB...,63
163964,to our email newsletter to hear about new eBo...,51


### 4 - Transformer le dataset en séquence « one hot encodées »

In [44]:
from sklearn.preprocessing import OneHotEncoder

X_int = np.array([[car_to_int[ch] for ch in seq] for seq in df['x']], dtype=np.int32)



In [45]:
X_int

array([[23, 26, 29, ..., 28,  5, 12],
       [26, 29,  5, ...,  5, 12, 18],
       [29,  5, 18, ..., 12, 18, 30],
       ...,
       [ 5, 18, 84, ..., 32, 55, 12],
       [18, 84, 32, ..., 55, 12, 63],
       [84, 32, 18, ..., 12, 63, 51]], shape=(163966, 50), dtype=int32)

In [ ]:
enc = OneHotEncoder()
enc.fit(X_int)
df['x'] = X_int

In [35]:
df

,x,y
0,23,18
1,26,30
2,29,58
3,5,18
4,18,62
...,...,...
163961,30,55
163962,88,12
163963,5,63
163964,18,51


In [22]:
n_total = len(df['x'])
N_seq = n_total - decalage
split_idx = int(N_seq * 0.7)

train_df = df.iloc[:split_idx]
test_df = df.iloc[split_idx:]

print(train_df.shape, test_df.shape)

(114741, 2) (49225, 2)


In [23]:
X_train = np.array(train_df['x'].tolist())
y_train = np.array(train_df['y'].tolist())

X_test = np.array(test_df['x'].tolist())
y_test = np.array(test_df['y'].tolist())

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(114741,) (114741,)
(49225,) (49225,)


### 5 - Proposer et tester différentes architecture LSTM entrainées sur le dataset précédemment généré. 

In [32]:
lstm = tf.keras.Sequential()

# Couche d'entrée de 1 neuronnes
lstm.add(tf.keras.Input(shape=(1,1)))
# Couche LSTM
lstm.add(tf.keras.layers.LSTM(50, return_sequences=True))
lstm.add(tf.keras.layers.LSTM(50, return_sequences=True))
lstm.add(tf.keras.layers.LSTM(50))
# Couche de sortir dense de 1 neuronne sans fonction d'activation
lstm.add(tf.keras.layers.Dense(1, activation=None))

adam = tf.keras.optimizers.Adam()
lstm.compile(loss='mean_squared_error', optimizer=adam)

In [33]:
lstm.fit(X_train, y_train, epochs=100, verbose=1, batch_size=128)

Epoch 1/100


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_6_1/Cast:0", shape=(None,), dtype=float32). Expected shape (None, 1, 1), but input has incompatible shape (None,)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None,), dtype=int64)
  • training=True
  • mask=None
  • kwargs=<class 'inspect._empty'>